## Test de détection de disques / cercles

Le but ici est de différencier les pions non pas grâce à un code mais grâce à un socle de couleur.

*Modules Python requis : numpy et opencv-contrib-python*

In [ ]:
import numpy as np
from collections import Counter
import cv2 as cv
import sys

**IMPORTANT : Quand les cellules de test de détection vont s'exécuter, une fenêtre s'appelant "Results" va s'ouvrir. Il ne faut PAS fermer la fenêtre avec la croix sinon jupyter sera bloqué ! Il suffit d'avoir le focus sur la fenêtre et d'appuyer sur une touche, par exemple la barre espace (attention, parfois la fenêtre est bien là, mais pas au premier plan, ce qui rend la chose un peu fastidieuse malheureusement).**

In [ ]:
# Le detecteur de code Aruco
det = cv.aruco.ArucoDetector()

In [ ]:
# Affichage de fenetre
def display(title, img):
    cv.imshow(title, img)
    cv.waitKey(0) # Attente d'appui sur une touche
    cv.destroyAllWindows()


# Surbrillance des cercles
def highlightCircles(img, circles):
    for i in circles[0]:
            center = (i[0], i[1])
            radius = i[2]
            cv.circle(img, center, 1, (0, 100, 100), 3)
            cv.circle(img, center, radius, (255, 0, 255), 3)
 
    display("Detection des cercles", img)


Pour détecter les cercles, j'utilise la transformée de Hough, via une fonction qui renvoie les couples (centre, rayon) de tous les cercles détectés.

In [ ]:
def detCircles(img):
    
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.medianBlur(gray, 5)
    
    circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, 30, param1=80, param2=18, minRadius=5, maxRadius=20)    
    
    circles = np.uint16(np.around(circles))

    
    return circles
    

def locate(coord, color):
    print(color)

La fonction de redressage de l'image est toujours présente :

In [ ]:
# Recupere les 4 coins du plateau
def getBoardCorners(img):
    pts, ids, reject = det.detectMarkers(img)
    corners = []

    # On garde les coordonnees des Aruco dont l'id est 0
    for i in range(len(ids)):
        if ids[i][0] == 0:
            corners.append(pts[i][0].astype(int))


    # On trie les coordonnees pour avoir les Aruco en partant d'en haut a gauche dans le sens horaire

    # Tri horizontal
    for i in range(len(corners)):
        for j in range(len(corners)-i-1):
            if corners[j][0][0] > corners[j+1][0][0]: # s'il est + a droite, on swap
                aux = corners[j]
                corners[j] = corners[j+1]
                corners[j+1] = aux
    
    # Tri vertical
    if corners[0][0][1] > corners[1][0][1]: # s'il est + bas, on swap
        aux = corners[0]
        corners[0] = corners[1]
        corners[1] = aux

    if corners[2][0][1] > corners[3][0][1]:
        aux = corners[2]
        corners[2] = corners[3]
        corners[3] = aux

    # Arrangement final 
    aux = corners[1]
    corners[1] = corners[2]
    corners[2] = corners[3]
    corners[3] = aux

    # Seuls les coins du plateau nous interessent a present
    corners[0] = corners[0][0]
    corners[1] = corners[1][1]
    corners[2] = corners[2][2]
    corners[3] = corners[3][3]

    return corners


Pour détecter la couleur des cercles, je vérifie si le centre du disque correspond à un intervalle de couleur parmi ceux définis.

Les intervalles définis sont construits à partir de couleurs d'étalonnage présents sur l'image.

In [ ]:
def detColor(img):
    display("Image d'origine", img)

    # On recupere les coins
    boardCorners = getBoardCorners(img)

    # Correction de la perspective
    pts1 = np.float32(boardCorners)
    pts2 = np.float32([[5,5], [1125,5], [1125,745], [5,745]])
    M = cv.getPerspectiveTransform(pts1,pts2)
    img = cv.warpPerspective(img,M,(1135,755))


    # On recupere les nouveaux coins
    boardCorners = getBoardCorners(img)
    display("Perspective corrigee", img)

    # Contours du plateau
    cv.polylines(img, np.array([boardCorners]), True, (0,0,255), 2)
    display("Detection du plateau", img)

    # Contours (pour visualiser seulement)
    edges = cv.Canny(img, 80, 40)
    display("Detection des contours", edges)

    # Detection des cercles
    circles = detCircles(img)

    # Etalonnage
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    cal = {"green" : (hsv[155][45] - np.array([5, 40, 60]), hsv[155][45] + np.array([5, 40, 60])),
           "navy" : (hsv[210][45] - np.array([5, 40, 60]), hsv[210][45] + np.array([5, 40, 60])),
           "beige" : (hsv[270][45] - np.array([5, 40, 60]), hsv[270][45] + np.array([5, 40, 60])),
           "sky" : (hsv[335][45] - np.array([5, 40, 60]), hsv[335][45] + np.array([5, 40, 60]))}

    for c in circles[0]:
        color = hsv[c[1]][c[0]]

        if (color >= cal["green"][0]).all() and (color <= cal["green"][1]).all():
            locate(c, "green")
        elif (color >= cal["navy"][0]).all() and (color <= cal["navy"][1]).all():
            locate(c, "navy")
        elif (color >= cal["beige"][0]).all() and (color <= cal["beige"][1]).all():
            locate(c, "beige")
        elif (color >= cal["sky"][0]).all() and (color <= cal["sky"][1]).all():
            locate(c, "sky")
        else:
            raise Exception("couleur non reconnue")

    

    highlightCircles(img, circles)



In [ ]:


# Pour boucler sur toutes les images
dictImg =  {0: "img/photo/france.jpg",
            1: "img/photo/france-2.jpg"} 
dico = dictImg

Boucle principale :

In [ ]:

for k in dico: # Pour chaque image

    print("\n################\n")

    img = cv.imread(dico[k]) # Lecture de img
    print(dico[k])
    print("")

    detColor(img)
   
    

Bilan : la détection de couleur avec l'étalonnage marche super bien pour ces couleurs là.

La détection des disques est un peu plus capricieuse, peut-être car ils ont été découpés à la main et ne sont donc pas parfaits.

Maintenant, je vais tester avec des cercles.

In [ ]:
dictImg =  {2: "img/photo/cercle-1.jpg",
            3: "img/photo/cercle-2.jpg",
            4: "img/photo/cercle-3.jpg",} 
dico = dictImg

In [ ]:
def detColor(img):
    display("Image d'origine", img)

    # On recupere les coins
    boardCorners = getBoardCorners(img)

    # Correction de la perspective
    pts1 = np.float32(boardCorners)
    pts2 = np.float32([[5,5], [1125,5], [1125,745], [5,745]])
    M = cv.getPerspectiveTransform(pts1,pts2)
    img = cv.warpPerspective(img,M,(1135,755))


    # On recupere les nouveaux coins
    boardCorners = getBoardCorners(img)
    display("Perspective corrigee", img)

    # Contours du plateau
    cv.polylines(img, np.array([boardCorners]), True, (0,0,255), 2)
    display("Detection du plateau", img)

    # Contours (pour visualiser seulement)
    edges = cv.Canny(img, 80, 40)
    display("Detection des contours", edges)

    # Detection des cercles
    circles = detCircles(img)

    # Etalonnage
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    cal = {"green" : (hsv[155][45] - np.array([5, 40, 60]), hsv[155][45] + np.array([5, 40, 60])),
           "navy" : (hsv[210][45] - np.array([5, 40, 60]), hsv[210][45] + np.array([5, 40, 60])),
           "beige" : (hsv[270][45] - np.array([5, 40, 60]), hsv[270][45] + np.array([5, 40, 60])),
           "sky" : (hsv[335][45] - np.array([5, 40, 60]), hsv[335][45] + np.array([5, 40, 60])),
           "feutre_vert" : (hsv[146][42] - np.array([5, 40, 60]), hsv[146][42] + np.array([5, 40, 60])),
           "feutre_rouge" : (hsv[210][40] - np.array([5, 40, 60]), hsv[210][40] + np.array([5, 40, 60]))}

    for c in circles[0]:
        color = hsv[c[1]][c[0]]
        borderColor = hsv[c[1]+c[2]][c[0]]

        if (color >= cal["green"][0]).all() and (color <= cal["green"][1]).all():
            locate(c, "green")
        elif (color >= cal["navy"][0]).all() and (color <= cal["navy"][1]).all():
            locate(c, "navy")
        elif (color >= cal["beige"][0]).all() and (color <= cal["beige"][1]).all():
            locate(c, "beige")
        elif (color >= cal["sky"][0]).all() and (color <= cal["sky"][1]).all():
            locate(c, "sky")
        elif (borderColor >= cal["feutre_vert"][0]).all() and (borderColor <= cal["feutre_vert"][1]).all():
            locate(c, "feutre_vert")
        elif (borderColor >= cal["feutre_rouge"][0]).all() and (borderColor <= cal["feutre_rouge"][1]).all():
            locate(c, "feutre_rouge")
        else:
            pass
            #raise Exception("couleur non reconnue")

    

    highlightCircles(img, circles)


Sensibilité beaucoup plus basse (param1 et param2):

In [ ]:
def detCircles(img):
    
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.medianBlur(gray, 5)
    
    circles = cv.HoughCircles(gray, cv.HOUGH_GRADIENT, 1, 30, param1=100, param2=30, minRadius=5, maxRadius=20)    
    
    circles = np.uint16(np.around(circles))

    
    return circles

Boucle principale :

In [ ]:

for k in dico: # Pour chaque image

    print("\n################\n")

    img = cv.imread(dico[k]) # Lecture de img
    print(dico[k])
    print("")

    detColor(img)
   
    

Ne pas faire attention aux couleurs pour l'instant !

A part ça, on voit que les cercles sont bien mieux détectés !

Maintenant, on va voir si c'est robuste quand le cercle n'est pas complet. Dans l'image suivante, j'ai ajouté 2 cercles quasi fermés, 2 cercles un peu moins fermés et 2 demi-cercles.

In [ ]:
dictImg =  {5: "img/photo/cercle-4.png"} 
dico = dictImg

In [ ]:

for k in dico: # Pour chaque image

    print("\n################\n")

    img = cv.imread(dico[k]) # Lecture de img
    print(dico[k])
    print("")

    detColor(img)
   
    

Comme on peut le voir (toujours sans regarder la couleur), même les demi-cercles sont parfaitement détectés.